In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('housingdata.csv')
df.head()
df.tail() #데이터셋 끝부분에도 데이터가 있는지 확인
df.shape
df.info() #문자열 데이터 없음 확인

In [ ]:
#결측치 유무 확인 (총 6개의 열에 각 20개씩 결측치가 존재함)
missing_data = df.isnull().sum()

missing_data

In [ ]:

#결측치 중앙값으로 대체하기
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
df_imputed = imputer.fit_transform(df)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
print(df_imputed)
print(np.sum(np.isnan(df_imputed)))

In [ ]:

#통계적 검정 확인(Shapiro-Wilk Test)
from scipy.stats import shapiro
stat, p_value = shapiro(df_imputed)
print("Test Statistic:", stat)
print("p-value:", p_value)


#IQR 이상치 탐지
Q1 = np.quantile(df_imputed, 0.25, axis=0)
Q3 = np.quantile(df_imputed, 0.75, axis=0)
IQR = Q3 - Q1

#IQR 1.5배 이상치 처리
outliers = ((df_imputed < (Q1 - 1.5 * IQR)) | (df_imputed > (Q3 + 1.5 * IQR))).any(axis=1)
print(outliers.sum())

#이상치 삭제
df_cleaned = pd.DataFrame(df_imputed[~outliers])

In [ ]:
df_cleaned.info()

In [8]:
#독립변수(X), 종속변수(y) 분리
X = df_cleaned.drop('MEDV' , axis=1)
y = df_cleaned['MEDV']

In [9]:
#train데이터와 test데이터 분리
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#특성 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
#모델 학습
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100)
}

results = {}

for name, model in models.items():
    # Training the model
    model.fit(X_train, y_train)
    
    # Predicting
    y_pred = model.predict(X_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
# 모델 학습 및 평가 (Pipeline 활용)
for name, model in models.items():
    # Pipeline 생성
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # 데이터 스케일링
        ('model', model)               # 모델 적용
    ])
    
    # 교차 검증 실행
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
    
    # 최종 학습 및 예측
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    # 평가 지표 계산
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # 결과 저장
    results[name] = {
        'RMSE': rmse,
        'R2': r2,
        'CV_R2_mean': cv_scores.mean(),
        'CV_R2_std': cv_scores.std()
    }

# 결과 출력
for name, result in results.items():
    print(f"\n{name} 결과:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"R2 Score: {result['R2']:.2f}")
    print(f"Cross-validation R2: {result['CV_R2_mean']:.2f} (+/- {result['CV_R2_std'] * 2:.2f})")

In [ ]:
# 결과 시각화
metrics = ['RMSE', 'R2', 'CV_R2_mean']
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, metric in enumerate(metrics):
    values = [results[model][metric] for model in results.keys()]
    axes[i].bar(results.keys(), values)
    axes[i].set_title(metric)
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()